# Collection of frequency and peak velocity parameters from all pipelines

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import json
import localdec

In [2]:
data_dir = "../SpeedData"
dist_col = 3
pls = pd.read_excel(data_dir + "/" +  "Datasets.xlsx",
                   sheet_name="Datasets")
pls["Pipeline"] = pls["Filename"].str.extract("^(.*)\sVelocity.*")
pls.set_index("Pipeline", inplace=True)
pls.head(12)

,PiplineNo,Filename,Sheet,Header,Skip,SpeedData1,SpeedData2,SpeedData3,Tool1,Tool2,Tool3,PredictorTool
Pipeline,,,,,,,,,,,,
CH-NC_508,1,CH-NC_508 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity \n(2019 BHGE GEMINI MFL),Velocity \n(2013 ROSEN AFD ),NaN,ROSEN AFD
LIV_PAT_457,2,LIV_PAT_457 Velocity Excursion Review.xlsm,Event Log & Velocity Listing,4,5,6,7.0,8.0,Velocity (2019 BHGE MFL),Velocity (2019 ROSEN MFL-C),Velocity (2019 ROSEN EMAT (wSC)),ROSEN EMAT (wSC)
HUN_NIC_762,3,HUN_NIC_762 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity \n(2016 BH Gemini HD MFL (wSC)),Velocity \n(2017 ROSEN AFD ),NaN,ROSEN AFD
LIV_COQ_323,4,LIV_COQ_323 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,9.0,Velocity (2019 BHGE MagneScan MFL),Velocity (2019 ROSEN MFL-A),Velocity (2019 ROSEN MFL-C),ROSEN MFL-C
NIC_FRA_610,5,NIC_FRA_610 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity \n(2013 BH MFL),Velocity (2016 Rosen MFL-C ),NaN,ROSEN MFL-C
NIC_PMA_610,6,NIC_PMA_610 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity (2019 BHI Gemini HD MFL (wSC)),Velocity (2016 ROSEN MFL-C),NaN,ROSEN MFL-C
ROE_TIL_914,7,ROE_TIL_914 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity (2013 BH MFL),Velocity\n (2016 ROSEN MFL-C),NaN,ROSEN MFL-C
TIL_BEN_323,8,TIL_BEN_323 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity \n(2013 BH MFL),Velocity (2017 Rosen MFL-C ),NaN,ROSEN MFL-C
TIL_FRA_508,9,TIL_FRA_508 Velocity Excursion Review.xlsm,Sheet1,4,5,7,8.0,NaN,Velocity \n(2013 BH MFL),Velocity (2016 Rosen MFL-C ),NaN,ROSEN MFL-C


### Read tool operating parameters

In [3]:
tool_params = pd.read_excel(data_dir + "/" +  "Datasets.xlsx",
                   sheet_name="Tools")
tool_params["ToolName"] = tool_params["ToolName"].str.upper()
tool_params.set_index("ToolName", inplace=True)
tool_params.head(5)

,Degrade,Blind,Peak,Comment
ToolName,,,,
ROSEN EMAT (WSC),2.5,4.0,4.00,Actual
ROSEN MFL-C,4.5,7.0,3.60,Good match
BHGE MFL,5.0,7.0,2.60,Okay match
BH MFL,5.0,7.0,2.60,Okay match
BHI GEMINI HD MFL (WSC),NaN,NaN,3.45,"Inverted match, weak"


### Create tool dictionary from pipeline spreadsheet

In [4]:
pipelines = dict()
colheads = [x.group(1) for x in [re.match("(^SpeedData\d)", col) for col in pls.columns] if x != None]
toolheads = [x.group(1) for x in [re.match("(^Tool\d)", col) for col in pls.columns] if x != None]
for pl in pls.index:
    pipeline = dict()
    tools=dict()
    i = 0
    for toolhead in toolheads:
        if pls.loc[pl, toolhead] == pls.loc[pl, toolhead]:
            tcol = {"column": int(pls.loc[pl, colheads[i]])}
            toolname = re.match("^Velocity\s*\(\d{4}(.*)\)$", pls.loc[pl, toolhead]).group(1).strip().upper()
            tools[toolname] = tcol
            i += 1
    pipeline["tools"] = tools
    pipeline["predictor"] = pls.loc[pl, "PredictorTool"].strip().upper()
    pipelines[pl] = pipeline

### Read data for each pipeline

In [5]:
for pipeline in pls.index:
    cols = [dist_col] + [tool[1]["column"] for tool in pipelines[pipeline]["tools"].items()]
    col_names = ["Distance"] +  [tool[0] for tool in pipelines[pipeline]["tools"].items()]
    data_skiprows = [i for i in range(4)]
    if not math.isnan(pls.loc[pipeline, "Skip"]):
        data_skiprows.append(int(pls.loc[pipeline, "Skip"]))
    print(pipeline)
    pl_raw = pd.read_excel(data_dir + "/" + pls.loc[pipeline, "Filename"],
                   sheet_name=pls.loc[pipeline, "Sheet"],
                   usecols=cols,
                   names = col_names,
                   skiprows=data_skiprows)
    pipelines[pipeline]["data"] = pl_raw

CH-NC_508
LIV_PAT_457
HUN_NIC_762
LIV_COQ_323
NIC_FRA_610
NIC_PMA_610
ROE_TIL_914
TIL_BEN_323
TIL_FRA_508
TIL_LNG_323
HUN_NIC_1067


## Loop through pipelines

In [6]:
dist_thresh = 25
pl_out = pd.DataFrame(columns=["Pipeline", "PredictorTool", "Length", "ExcursionCount", "ExcursionLength"])

#for pipeline in pls[pls["PredictorTool"] != "ROSEN AFD"].index:
for pipeline in pls.index:
    tool = pls.loc[pipeline, "PredictorTool"].upper()
    peak = tool_params.loc[tool, "Peak"]
    print("{} --> predictor: {}".format(pipeline, tool))
    pl_raw =  pipelines[pipeline]["data"]
    pl_length = pl_raw["Distance"].max() - pl_raw["Distance"].min()
    
    # unpivot data and create temporary dataframe
    pl = pd.melt(pl_raw,
                    id_vars="Distance",
                    var_name="Tool",
                    value_name="Velocity")
    pl["Tool"] = pl["Tool"].astype('category')
    pl.loc[pl["Tool"] == tool, "Excursion"] = 0 # reset
    localdec.mark_excursion(tool, peak, dist_thresh, pl)
    exc_count = pl.groupby("Tool").get_group(tool)["Excursion"].sum()
    exc_length = pl.groupby("Tool").get_group(tool)["Length"].mean()
    exc_rate = exc_count/(pl_length/1000)  # excursions per km'''
    pl_out = pl_out.append({"Pipeline": pipeline,
                   "PredictorTool": pls.loc[pipeline, "PredictorTool"],
                   "Length": pl_length,
                   "ExcursionCount": exc_count,
                   "ExcursionLength": exc_length
                   }, ignore_index=True)
    print("distance: {:.1f}km, number excursions: {}, rate: {:.3f} excursions/km, avg length: {}".format(pl_length/1000, exc_count, exc_rate, exc_length))

CH-NC_508 --> predictor: ROSEN AFD
distance: 9.0km, number excursions: 2.0, rate: 0.221 excursions/km, avg length: 4482.0938914647
LIV_PAT_457 --> predictor: ROSEN EMAT (WSC)
distance: 29.8km, number excursions: 8.0, rate: 0.269 excursions/km, avg length: 5.322625000000471
HUN_NIC_762 --> predictor: ROSEN AFD
distance: 56.5km, number excursions: 58.0, rate: 1.027 excursions/km, avg length: 914.8400248793105
LIV_COQ_323 --> predictor: ROSEN MFL-C
distance: 34.7km, number excursions: 52.0, rate: 1.500 excursions/km, avg length: 117.18121153846141
NIC_FRA_610 --> predictor: ROSEN MFL-C
distance: 24.3km, number excursions: 39.0, rate: 1.602 excursions/km, avg length: 139.06492307692292
NIC_PMA_610 --> predictor: ROSEN MFL-C
distance: 5.0km, number excursions: 10.0, rate: 2.015 excursions/km, avg length: 112.52282270000003
ROE_TIL_914 --> predictor: ROSEN MFL-C
distance: 12.8km, number excursions: 10.0, rate: 0.783 excursions/km, avg length: 155.15199999999996
TIL_BEN_323 --> predictor: ROS

In [7]:
pl_out.set_index("Pipeline", inplace=True)
pl_out

,PredictorTool,Length,ExcursionCount,ExcursionLength
Pipeline,,,,
CH-NC_508,ROSEN AFD,9037.285407,2.0,4482.093891
LIV_PAT_457,ROSEN EMAT (wSC),29777.100000,8.0,5.322625
HUN_NIC_762,ROSEN AFD,56451.747817,58.0,914.840025
LIV_COQ_323,ROSEN MFL-C,34670.809000,52.0,117.181212
NIC_FRA_610,ROSEN MFL-C,24344.515000,39.0,139.064923
NIC_PMA_610,ROSEN MFL-C,4963.774098,10.0,112.522823
ROE_TIL_914,ROSEN MFL-C,12763.874000,10.0,155.152000
TIL_BEN_323,ROSEN MFL-C,5892.920000,5.0,118.603600
TIL_FRA_508,ROSEN MFL-C,9706.206000,13.0,118.902000


In [8]:
pl_out.to_excel(excel_writer = data_dir + "/" +  "SpeedModelOutput.xlsx",
                   sheet_name="PipelineEstimates")